In [1]:
import pandas as pd
import ktrain
from ktrain import text
import sagemaker

In [2]:
sagemaker.s3.S3Downloader.download(local_path="./model/", s3_uri="s3://sagemaker-us-east-1-652081700929/final-distilbert-regression-800k-2020-06-15-07-04-51-437/output/model.tar.gz")

In [ ]:
sagemaker.s3.S3Downloader.download(local_path="./ft_model/", s3_uri="s3://sagemaker-us-east-1-652081700929/final-distilbert-regression-800k-2020-06-15-07-04-51-437/output/model.tar.gz")

In [3]:
%%sh
cd model
tar -xf model.tar.gz

In [4]:
!rm -rf model/model.tar.gz

In [5]:
predictor = ktrain.load_predictor("model")

In [6]:
predictor.predict("this place is amazing!")

/Users/spenceradams/.local/share/virtualenvs/Yelp-ds-xpajUQwb/lib/python3.7/site-packages/ktrain/text/preprocessor.py:508: UserWarning: Task is being treated as TEXT REGRESSION because class_names argument was not supplied. If this is incorrect, supply class_names argument.
  'class_names argument was not supplied. ' + \


5.074088

In [8]:
%%timeit
predictor.predict("this place is amazing!")

658 ms ± 26.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [9]:
# ^^ 67.6 ms on gpu

In [10]:
from transformers import AutoTokenizer, AutoConfig, TFAutoModelForSequenceClassification

In [11]:
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased', use_fast=True)
model = TFAutoModelForSequenceClassification.from_pretrained("model")

In [12]:
def predict(text):
    return model(tokenizer.encode(text, return_tensors='tf', max_length=512))[0].numpy()[0][0]

In [13]:
%%timeit
predict("this place is amazing!")

94.2 ms ± 3.94 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [14]:
# ^^ 62 ms on gpu

In [15]:
predictor.predict("this place is amazing!")

5.074088

In [16]:
predict("this place is amazing!")

5.074088

In [18]:
from tqdm.notebook import tqdm

In [19]:
val = pd.read_csv("s3://yelp-dataset-pt-9/spencer/data/sentiment/en/val.csv")

In [20]:
val.head()

,text,stars
0,Really great local auto repair spot... really ...,5.0
1,Best Oyster House in Toronto. Great variety a...,5.0
2,We thoroughly enjoy eating here. We go here at...,5.0
3,OMG!!! So good!!!! I was in AZ for work and I...,5.0
4,We are here once over a year ago before we mov...,1.0


In [21]:
tqdm.pandas()

In [23]:
from sklearn.metrics import mean_absolute_error

In [26]:
from IPython.display import clear_output

In [37]:
import csv

In [51]:
import time

In [70]:
pred_df = pd.read_csv("preds.csv", header=None, names=['idx', 'text', 'stars', 'pred'])

In [71]:
pred_df.shape

(300823, 4)

In [76]:
pred_df = pd.read_csv("preds.csv", header=None, names=['idx', 'text', 'stars', 'pred'])
pred_df.shape

(11, 4)

In [77]:
pred_df = pd.read_csv("preds-copy.csv", header=None, names=['idx', 'text', 'stars', 'pred'])
pred_df.shape

(300823, 4)

In [78]:
true = pred_df['stars'].tolist()
preds = pred_df['pred'].tolist()

In [79]:
mean_absolute_error(true, preds)

0.32672610572889627

In [74]:
def predict_and_print(df):
    with open('preds.csv', 'w', newline='') as csvfile:
        spamwriter = csv.writer(csvfile)
        
        for idx, text, stars in tqdm(df.itertuples(), total=len(df)):
            pred = float(predict(text))
            preds.append(pred)
            true.append(stars)
            spamwriter.writerow([idx, text, stars, pred])
            print(mean_absolute_error(true, preds), end="\r")
                
            
    return preds

In [80]:
preds = predict_and_print(val.iloc[300823:])

0.32834940420982933


KeyboardInterrupt: 